In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import requests

NGROK_DOMAIN = "http://eez115.ece.ust.hk:5000/"
# NGROK_DOMAIN = "http://localhost:8080/"
session_id = 227194704


response = requests.post(f"{NGROK_DOMAIN}/get_paf_result", json={"session_id": session_id})
response = response.json()
response_data = response.get("responses", {})
conversation_history = response_data.pop("conversation_history", None)
table = pd.DataFrame.from_dict(response_data, orient='index')

In [3]:
conversation_history

[{'role': 'Grace', 'utterance': '你好啊！我喺 Grace。你今日覺得點啊？'},
 {'role': 'Patient', 'utterance': 'hi'},
 {'role': 'Grace', 'utterance': '請問你有冇咳？'},
 {'role': 'Patient', 'utterance': 'no'},
 {'role': 'Grace', 'utterance': '冇。請問有冇痰？'},
 {'role': 'Patient', 'utterance': 'no'},
 {'role': 'Grace', 'utterance': '請問你過去嗰三個月有冇無端端輕左？'},
 {'role': 'Patient', 'utterance': 'yes about 3kg'},
 {'role': 'Grace', 'utterance': '胃口有冇差到？'},
 {'role': 'Patient', 'utterance': 'yes'},
 {'role': 'Grace', 'utterance': '收到。你有冇特別嘅嘢唔食㗎？'},
 {'role': 'Patient', 'utterance': 'i like chicken'},
 {'role': 'Grace', 'utterance': '好多謝你嘅時間，我已經將你所有嘅答案記錄曬。'}]

In [4]:
table

,patient_answer,question_asked,session_id,slot_name,slot_value
appetite_loss,yes,胃口有冇差到？,297706618,appetite_loss,yes
cough_existence,no,請問你有冇咳？,297706618,cough_existence,no
food_preference,i like chicken,你有冇特別嘅嘢唔食㗎？,297706618,food_preference,no preference
sputum_existence,no,請問有冇痰？,297706618,sputum_existence,no.
weight_loss_amount,yes about 3kg,請問你過去嗰三個月有冇無端端輕左？,297706618,weight_loss_amount,1-5kg
weight_loss_existance,yes about 3kg,請問你過去嗰三個月有冇無端端輕左？,297706618,weight_loss_existance,yes


In [5]:
table.reset_index(drop=True)[["session_id", "slot_name", "question_asked","slot_value"]]

,session_id,slot_name,question_asked,slot_value
0,297706618,appetite_loss,胃口有冇差到？,yes
1,297706618,cough_existence,請問你有冇咳？,no
2,297706618,food_preference,你有冇特別嘅嘢唔食㗎？,no preference
3,297706618,sputum_existence,請問有冇痰？,no.
4,297706618,weight_loss_amount,請問你過去嗰三個月有冇無端端輕左？,1-5kg
5,297706618,weight_loss_existance,請問你過去嗰三個月有冇無端端輕左？,yes


In [6]:
table.slot_name.unique()

array(['appetite_loss', 'cough_existence', 'food_preference',
       'sputum_existence', 'weight_loss_amount', 'weight_loss_existance'],
      dtype=object)

In [7]:
from fill_PAF_table import fill_paf_table

tab = fill_paf_table(response.get("responses", {}))
tab

,PAF_Question,PAF_Result
1,Coughing 請問有冇咳？,☑️ No \t ☐ Yes
2,Sputum 請問有冇痰？,☑️ No \t ☐ Yes
2.1,\t – \t Sputum Amount 有痰的話，有幾多量？,N.A.
2.2,\t – \t Sputum Color 痰顏色係咩？,N.A.
3,Weight Loss 有冇體重下降？,☐ No \t ☑️ Yes
3.1,\t – \t Weight Loss Amount 輕咗幾多？,1-5kg
4,Appetite Loss 胃口差别？,☐ No \t ☑️ Yes
5,Food Preference 咩唔食？,no preference


In [8]:
session_info = requests.post(f"{NGROK_DOMAIN}/get_session_info", json={"session_id": session_id})
session_info = session_info.json()

In [9]:
session_info

{'responses': {'curr_question_id': 'initial_status',
  'grace_uttrs_history': ['initial_status'],
  'history': ['initial_status'],
  'q_tracker': ['initial_status'],
  'session_id': 297706618,
  'user_uttrs_history': []}}